In [21]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack import Pipeline
from haystack.components.routers import ConditionalRouter
import xml.etree.ElementTree as ET


import os
from getpass import getpass
from haystack_integrations.components.generators.amazon_bedrock import AmazonBedrockGenerator


from datasets import load_dataset
from haystack import Document
from inputs import matrix, operational_management_skillset_explanation, NOTES_PER_EXERCISE

In [22]:
def generate_skill_xml_string(skill_name, skill_value, skill_description):
    root = ET.Element("Skill")
    name = ET.SubElement(root, "Name")
    name.text = sskill_name
    value = ET.SubElement(root, "Value")
    value.text = str(skill_value)    
    description = ET.SubElement(root, "Description")
    for item in skill_description:
        point = ET.SubElement(description, "Point")
        point.text = item
    xml_str = ET.tostring(root, encoding='unicode', method='xml')
    return xml_str

In [24]:
skill_name = 'Kunden verstehen'
skill_value = matrix[skill_name]
skill_description = operational_management_skillset_explanation[skill_name]

In [25]:
skill_xml_string = generate_skill_xml_string(skill_name, skill_value, skill_description)

In [26]:
notes = ""
for e in NOTES_PER_EXERCISE:
    notes += e + "\n"
    notes += NOTES_PER_EXERCISE[e]

In [36]:
template = """
You are an expert HR consultant specializing in evaluating human psychology and behavior during candidate assessments. Your task is to analyze the alignment between qualitative observation notes and a quantitative skill evaluation for a job candidate.

Context:
- Observation Notes: Detailed qualitative observations from the assessment process
- Numerical Skill Evaluation: A rating of the candidate's skills on a scale from 0 to 4
  0 = Not observed/Not applicable
  1 = Poor performance
  2 = Good/Expected performance
  3 = Very good performance
  4 = Excellent performance
- Skill Description: Detailed explanation of the skill's attributes and expectations

Instructions:
1. Carefully read the observation notes and skill description.
2. For each attribute of the skill:
   a. Identify specific notes that support or contradict the numerical rating.
   b. Evaluate the strength and relevance of each piece of evidence.
   c. Provide supporting and opposing evidence for every point of the skill, even if one type of evidence is weaker or less prevalent.
3. Determine if the overall observations align with the provided numerical evaluation.
4. Provide your analysis in the structured format below.

Output Format:
{
  "skill_name": "Name of the skill being evaluated",
  "numerical_rating": "The provided numerical rating (0-4)",
  "skill_attributes": [
    {
      "attribute": "Specific attribute of the skill",
      "supporting_evidence": [
        {
          "citation": "Exact quote from observation notes",
          "explanation": "Why this supports the rating"
        }
      ],
      "opposing_evidence": [
        {
          "citation": "Exact quote from observation notes",
          "explanation": "Why this contradicts the rating"
        }
      ],
      "attribute_analysis": "Your assessment of how well the evidence aligns with the numerical rating for this specific attribute"
    }
  ],
  "overall_analysis": "Your overall assessment of whether the observations align with the numerical rating, including any nuances or potential discrepancies across all attributes",
  "confidence": "High/Medium/Low - Your level of confidence in this analysis based on the available evidence"
}

Important:
- Be objective and thorough in your analysis.
- Consider the context and specific requirements of the skill.
- If there's insufficient evidence to make a determination, state this clearly.

Observation Notes:
{{observation_notes}}

Skill Description and Numerical Rating:
{{skill}}

Please provide your analysis based on the instructions and format above.
"""
promptbuilder = PromptBuilder(template)

In [37]:
generator = AmazonBedrockGenerator(model="anthropic.claude-3-5-sonnet-20240620-v1:0", max_length=4000, truncate=False)

In [38]:
pipe = Pipeline()

pipe.add_component("prompt_builder", promptbuilder)
pipe.add_component("generator", generator)
pipe.connect("prompt_builder", "generator")

🚅 Components
  - prompt_builder: PromptBuilder
  - generator: AmazonBedrockGenerator
🛤️ Connections
  - prompt_builder.prompt -> generator.prompt (str)

In [39]:
response = pipe.run(data={"prompt_builder": {"observation_notes": notes, "skill": skill_xml_string}})

In [35]:
print(response['generator']['replies'][0])

{
  "skill_name": "Kunden verstehen",
  "numerical_rating": 3.5,
  "supporting_evidence": [
    {
      "citation": "Vermittelt eine hohe Kundenorientierung und bemisst den Erfolg seiner Vorhaben an den Kundenmehrwert sowie an der Kundenbindung",
      "explanation": "This directly supports the skill of understanding customers and prioritizing customer value."
    },
    {
      "citation": "Fokussiert sich auf die Erfolgskommunikation und Prozessoptimierung, um den gestellten Business Case zu lösen",
      "explanation": "This aligns with optimizing work processes to deliver better results for customers."
    },
    {
      "citation": "Zeigt grundsätzlich eine hohe Offenheit, mit verschiedenen Stakeholdern zu kommunizieren, um die Idee des Veränderungsvorhabens zunächst mit verschiedenen Parteien zu besprechen und dadurch Risiken abzubauen",
      "explanation": "This demonstrates a proactive approach to understanding and addressing customer needs across different stakeholders."
    